In [1]:
import os
#os.environ["TF_GPU_THREAD_MODE"] = "gpu_private"

from pprint import pprint
import numpy as np
import tensorflow as tf
import keras
from tensorflow.python.client import device_lib
from keras.applications import MobileNetV2
from keras.layers import Dense, GlobalAveragePooling2D, Input
from keras.models import Model
from keras.optimizers import Adam
from keras.utils import Sequence


gpu_options = tf.compat.v1.GPUOptions(per_process_gpu_memory_fraction=1)
sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(gpu_options=gpu_options))

print("TensorFlow version:", tf.__version__)
print("GPU Available:", tf.test.is_gpu_available())

# Get detailed device information
print(device_lib.list_local_devices())

2025-01-16 11:57:12.581091: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-16 11:57:12.590665: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-01-16 11:57:12.601779: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-01-16 11:57:12.605485: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-16 11:57:12.615260: I tensorflow/core/platform/cpu_feature_guar

TensorFlow version: 2.17.0
Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
GPU Available: True
[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 2473724590176199256
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 3983474688
locality {
  bus_id: 1
  links {
  }
}
incarnation: 16284674807100838258
physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 3050 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6"
xla_global_id: 416903419
]


I0000 00:00:1737008837.017154  868906 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1737008837.019018  868906 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1737008837.114051  868906 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1737008837.115236  868906 cuda_executor.cc:1015] successful NUMA node read from SysFS ha

In [2]:
memmap_path='/mnt/Velocity_Vault/Autofocus/Dataset_Storage/'
train_size=355
patch_num=12
test_size=47
num_classes=49

train_dataset = np.memmap(memmap_path+"train_dataset", dtype=np.float64, mode='r', shape=(patch_num*train_size,128, 128,98))
train_truth = np.memmap(memmap_path+"train_truth_one_hot", dtype=int, mode='r', shape=(patch_num*train_size,num_classes))


In [3]:
# memmap_path='/mnt/Velocity_Vault/Autofocus/Dataset_Storage/'
# train_size=355
# patch_num=12
# ratio=5
# test_size=47
# num_classes=49

# org_train_dataset = np.memmap(memmap_path+"train_dataset", dtype=np.float64, mode='r', shape=(patch_num*train_size,128, 128,98))
# org_train_truth = np.memmap(memmap_path+"train_truth_one_hot", dtype=int, mode='r', shape=(patch_num*train_size,num_classes))

# decreased_size=(train_size)//ratio

# train_dataset = np.memmap(memmap_path+"small_train_dataset", dtype=np.float64, mode='w+', shape=(patch_num*decreased_size,128, 128,98))
# train_truth = np.memmap(memmap_path+"small_train_truth_one_hot", dtype=int, mode='w+', shape=(patch_num*decreased_size,num_classes))

# train_dataset=org_train_dataset[:patch_num*decreased_size]
# train_truth=org_train_truth[:patch_num*decreased_size]

# train_dataset.flush()
# train_truth.flush()

# train_size=decreased_size





In [4]:
print(train_dataset.shape)
print(train_truth.shape)

(4260, 128, 128, 98)
(4260, 49)


In [5]:
memmap_path='/mnt/Velocity_Vault/Autofocus/Dataset_Storage/'
test_size=47
num_classes=49

test_dataset = np.memmap(memmap_path+"test_dataset", dtype=np.float64, mode='r', shape=(patch_num*test_size,128, 128,98))
test_truth = np.memmap(memmap_path+"test_truth", dtype=int, mode='r', shape=(patch_num*test_size))

In [6]:
def create_model(input_shape=(128, 128, 98), num_classes=49):
    base_model = MobileNetV2(input_shape=input_shape, include_top=False, weights=None)
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    logits = Dense(num_classes, activation=None)(x)  # Output 49 logits
    model = Model(inputs=base_model.input, outputs=logits)
    return model

def ordinal_loss(y_true, y_pred):
    l2_loss = tf.reduce_mean(tf.square(y_pred - y_true), axis=None)
    return l2_loss

model = create_model()

# Compile the model
model.compile(
    optimizer=Adam(learning_rate=1e-3, beta_1=0.5, beta_2=0.999),
    loss=ordinal_loss
)


/mnt/Installation/Virtual_Environment/Conda3_12/lib/python3.12/site-packages/keras/src/applications/mobilenet_v2.py:210: UserWarning: This model usually expects 1 or 3 input channels. However, it was passed an input_shape with 98 input channels.
  input_shape = imagenet_utils.obtain_input_shape(
I0000 00:00:1737008837.185030  868906 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1737008837.186660  868906 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1737008837.188250  868906 cuda_executor.cc:1015] successful NUMA node read from SysFS had n

In [7]:
# model_config = model.get_config()
# print(model_config)
# model.summary()

In [8]:
set_size=train_size*patch_num
batch_size = 3
global_steps = 10000
steps_per_epoch = set_size // batch_size  
epochs = global_steps // steps_per_epoch

print(set_size)
print(batch_size)
print(steps_per_epoch)
print(epochs)

4260
3
1420
7


In [9]:
import numpy as np   

class DataGenerator(Sequence):
    def __init__(self, x_set, y_set, batch_size):
        self.x, self.y = x_set, y_set
        self.batch_size = batch_size

    def __len__(self):
        return int(np.ceil(len(self.x) / float(self.batch_size)))

    def __getitem__(self, idx):
        batch_x = self.x[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_y = self.y[idx * self.batch_size:(idx + 1) * self.batch_size]
        return batch_x, batch_y

train_gen = DataGenerator(train_dataset, train_truth, batch_size)


#test_gen = DataGenerator(test_dataset, test_truth, batch_size)


model.fit(train_gen, epochs=epochs,verbose=1)

model_path='/mnt/Velocity_Vault/Autofocus/Model/Model_V1.keras'
model.save(model_path)  # Save in SavedModel format




TypeError: unhashable type: 'memmap'

In [10]:
# def ordinal_loss(y_true, y_pred):
#     l2_loss = tf.reduce_mean(tf.square(y_pred - y_true), axis=None)
#     return l2_loss

# from tensorflow.keras.models import load_model
# model_path='/mnt/Velocity_Vault/Autofocus/Model/Model_V1.keras'
# model=load_model(model_path,custom_objects={'ordinal_loss': ordinal_loss})

In [11]:
#print(test_dataset)
print(test_dataset.shape)

(564, 128, 128, 98)


In [48]:

predictions=np.zeros((47 * 12, 49), dtype=np.float64)

# printrobabilities = tf.nn.softmax(logits, axis=-1).numpy()

for i in range(12):
    test_temp=test_dataset[i*47:(i+1)*47]
    pred_temp=model.predict(test_temp)
    for j in range(47):
        predictions[i*47+j]=pred_temp[j]

print(predictions.shape)
pprint(predictions)




#predictions=model.predict(test_dataset)


2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
(564, 49)
array([[5.16122859e-03, 2.91819498e-02, 2.83615440e-02, ...,
        9.31493923e-05, 3.44047759e-04, 2.08488258e-04],
       [5.13715390e-03, 2.93165464e-02, 2.84816865e-02, ...,
        9.43924751e-05, 3.45634908e-04, 1.95445376e-04],
       [5.13672549e-03, 2.93174814e-02, 2.84823049e-02, ...,
        9.43182022e-05, 3.45553883e-04, 1.95358531e-04],
       ...,
       [5.07344212e-03, 2.95772925e-02, 2.86925826e-02, ...,
        9.36429933e-05, 3.37831530e-04, 1.70583720e-04],
       [5.09246625e-03, 2.94827893e-02, 2.86200698

In [19]:
def get_max_positions(array):
    # Find the index of the max value along each row
    return np.argmax(array, axis=1)

pred_truth=get_max_positions(predictions)

In [20]:
def calculate_accuracy(array1, array2):
    # Check if both arrays are the same shape
    if array1.shape != array2.shape:
        raise ValueError("Arrays must have the same shape")
    
    # Compare the arrays and calculate the accuracy
    correct_predictions = np.sum(array1 == array2)
    total_elements = array1.size
    
    accuracy = correct_predictions / total_elements
    return accuracy*100

pred_test=calculate_accuracy(pred_truth,test_truth)

print("Exact Correctness \n")

print("\nPredicted Truth vs Ground Truth - ",pred_test)



Exact Correctness 


Predicted Truth vs Ground Truth -  9.042553191489363


In [52]:
pprint(pred_truth)

array([23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23,
       23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23,
       23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23,
       23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23,
       23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23,
       23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23,
       23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23,
       23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23,
       23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23,
       23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23,
       23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23,
       23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23,
       23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23,
       23, 23, 23, 23, 23

In [21]:
def calculate_deviation(predictions, ground_truth):
    assert len(predictions) == len(ground_truth), "Prediction and ground truth lists must have the same length."
    
    new_array = (ground_truth - predictions)
    ma=np.max(new_array)
    av=np.mean(new_array)
    
    # Compute squared errors
    squared_errors = [(p - gt) ** 2 for p, gt in zip(predictions, ground_truth)]
    
    # Calculate mean of squared errors
    mse = sum(squared_errors) / len(ground_truth)
    
    return ma,av,mse

ma,av,mse=calculate_deviation(pred_truth,test_truth)


print("Deviation Correctness \n")

print('\nMax Deviation - ',ma)
print('\nAvg Deviation - ',av)
print('\nMean Square Deviation - ',mse)

Deviation Correctness 


Max Deviation -  1

Avg Deviation -  -9.358156028368795

Mean Square Deviation -  133.01418439716312
